# Projekt: Algorytm genetyczny dla problemu komiwojażera (TSP)

**Cel notebooka**: krok-po-kroku implementacja algorytmu genetycznego (GA) dla TSP, wyjaśnienia koncepcyjne, kod w Pythonie z polskimi nazwami zmiennych oraz przykłady eksperymentów i wykresów.

Notebook zawiera:
- opis problemu i podejścia,
- implementację operatorów (krzyżowania, mutacje, selekcje),
- prosty lokalny poprawiacz (swap, 2-opt, insertion),
- funkcję `run_ga(...)` do uruchamiania algorytmu z parametrami,
- przykładowe uruchomienia i wykresy konwergencji,
- instrukcję jak prowadzić eksperymenty (zapisywanie wyników, powtórzenia).

> Uwaga: kod napisany prostym, czytelnym stylem, z polskimi nazwami funkcji i zmiennych — bez użycia gotowych bibliotek algorytmicznych.


## 1. Problem komiwojażera (TSP)

Problem: znaleźć najkrótszą trasę, która odwiedza każde miasto dokładnie raz i wraca do punktu początkowego.

Reprezentacja w GA: permutacja długości `n` (lista indeksów miast `0..n-1`).

Funkcja celu: długość zamkniętej trasy (suma odległości między kolejnymi miastami, z powrotem do startu).


## 2. Import bibliotek i wczytanie danych

Poniżej komórka z niezbędnymi importami i przykładem wczytania macierzy odległości z pliku Excel. W twoim środowisku podaj ścieżkę do pliku (np. `Dane_TSP_127.xlsx`).

In [2]:
# Importy
import random
import math
import time
import json
from copy import deepcopy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



sciezka = "Dane_TSP_127.xlsx"
macierz = pd.read_excel(sciezka, header=0, index_col=0).values
print("Rozmiar macierzy:", macierz.shape)



Rozmiar macierzy: (127, 127)


## 3. Funkcja celu i funkcje pomocnicze

Funkcja `dlugosc_trasy` oblicza długość zamkniętej trasy. Dodatkowo funkcja do generowania losowej trasy i wydruku trasy.

In [3]:
def dlugosc_trasy(trasa, dist):
    """Oblicza długość zamkniętej trasy (wraca do punktu startu)."""
    s = 0.0
    for i in range(len(trasa)):
        a = trasa[i]
        b = trasa[(i+1) % len(trasa)]
        s += dist[a, b]
    return s

def losowa_trasa(n):
    t = list(range(n))
    random.shuffle(t)
    return t

# Krótki test
t = losowa_trasa(n_miast)
print('Przykładowa trasa:', t)
print('Długość trasy (test):', dlugosc_trasy(t, macierz))

Przykładowa trasa: [1, 9, 8, 5, 10, 2, 3, 7, 4, 0, 11, 6]
Długość trasy (test): 14252.957999560236


## 4. Ruchy lokalne (lokalne poprawianie)

Implementujemy trzy proste ruchy: `swap`, `two_opt` (inwersja fragmentu) oraz `insertion` (wyjęcie i wstawienie). Te ruchy używamy w opcjonalnym lokalnym poprawiaczu, który może poprawić każdego potomka.

In [ ]:
def ruch_swap(trasa):
    a,b = random.sample(range(len(trasa)), 2)
    tr = trasa.copy()
    tr[a], tr[b] = tr[b], tr[a]
    return tr

def ruch_two_opt(trasa):
    a,b = sorted(random.sample(range(len(trasa)),2))
    tr = trasa.copy()
    tr[a:b+1] = list(reversed(tr[a:b+1]))
    return tr

def ruch_insertion(trasa):
    a,b = random.sample(range(len(trasa)),2)
    tr = trasa.copy()
    city = tr.pop(a)
    tr.insert(b, city)
    return tr

def lokalne_poprawienie(trasa, dist, ruchy=['swap','two_opt','insertion'], proby=10):
    najlepsza = trasa.copy()
    najlepsza_w = dlugosc_trasy(najlepsza, dist)
    for _ in range(proby):
        wybor = random.choice(ruchy)
        if wybor == 'swap':
            kand = ruch_swap(najlepsza)
        elif wybor == 'two_opt':
            kand = ruch_two_opt(najlepsza)
        else:
            kand = ruch_insertion(najlepsza)
        w = dlugosc_trasy(kand, dist)
        if w < najlepsza_w:
            najlepsza, najlepsza_w = kand, w
    return najlepsza

# Test lokalnego poprawiacza
t0 = losowa_trasa(n_miast)
print('start dlugosc:', dlugosc_trasy(t0, macierz))
t1 = lokalne_poprawienie(t0, macierz, proby=20)
print('po lokalnym poprawieniu dlugosc:', dlugosc_trasy(t1, macierz))

## 5. Krzyżowania (PMX, OX, CX)

Implementujemy trzy klasyczne krzyżowania dla permutacji: PMX, Order Crossover (OX) oraz Cycle Crossover (CX).

In [ ]:
def pmx(p1, p2):
    n = len(p1)
    a,b = sorted(random.sample(range(n),2))
    child = [-1]*n
    child[a:b+1] = p1[a:b+1]
    present = set(child[a:b+1])
    pos_in_p2 = {val:i for i,val in enumerate(p2)}
    for i in range(a,b+1):
        val = p2[i]
        if val in present:
            continue
        cur = i
        mapped = val
        while True:
            val1 = p1[cur]
            cur = pos_in_p2[val1]
            if child[cur] == -1:
                child[cur] = mapped
                present.add(mapped)
                break
    it = (v for v in p2 if v not in present)
    for i in range(n):
        if child[i]==-1:
            child[i]=next(it)
    return child

def ox(p1, p2):
    n = len(p1)
    a,b = sorted(random.sample(range(n),2))
    child = [-1]*n
    child[a:b+1] = p1[a:b+1]
    present = set(child[a:b+1])
    idx = (b+1)%n
    for v in p2:
        if v in present: continue
        child[idx]=v
        idx = (idx+1)%n
    return child

def cycle_crossover(p1, p2):
    n = len(p1)
    child = [-1]*n
    remaining = set(range(n))
    pos_in_p1 = {val:i for i,val in enumerate(p1)}
    cycle_num = 0
    while remaining:
        start = next(iter(remaining))
        idx = start
        indices = []
        while True:
            indices.append(idx)
            remaining.remove(idx)
            val = p2[idx]
            idx = pos_in_p1[val]
            if idx == start:
                break
        if cycle_num % 2 == 0:
            for i in indices: child[i] = p1[i]
        else:
            for i in indices: child[i] = p2[i]
        cycle_num += 1
    return child

# Test krzyżowań
a = losowa_trasa(n_miast)
b = losowa_trasa(n_miast)
print('parent1', a)
print('parent2', b)
print('pmx   ', pmx(a,b))
print('ox    ', ox(a,b))
print('cx    ', cycle_crossover(a,b))

## 6. Mutacje i selekcje

Mutacja: zamiana dwóch genów (swap). Selekcje: ruletka, turniej, rank (rank-based).

In [ ]:
def mutacja_swap(trasa):
    return ruch_swap(trasa)

def selekcja_ruletka(pop, fitness):
    fit = np.array(fitness)
    probs = fit/fit.sum()
    idx = np.random.choice(len(pop), p=probs)
    return deepcopy(pop[idx])

def selekcja_turniej(pop, fitness, k=3):
    idxs = random.sample(range(len(pop)), k)
    best = max(idxs, key=lambda i: fitness[i])
    return deepcopy(pop[best])

def selekcja_rank(pop, fitness):
    ranks = np.argsort(np.argsort(fitness))
    probs = (ranks + 1) / ((ranks + 1).sum())
    idx = np.random.choice(len(pop), p=probs)
    return deepcopy(pop[idx])

selekcje = {'ruletka': selekcja_ruletka, 'turniej': selekcja_turniej, 'rank': selekcja_rank}
krzyzowania = {'pmx': pmx, 'ox': ox, 'cx': cycle_crossover}

## 7. Główny algorytm genetyczny `run_ga`

Funkcja `run_ga` przyjmuje parametry: rozmiar populacji, liczbę generacji, prawdopodobieństwo mutacji, metodę selekcji, metodę krzyżowania, elita, czy używać lokalnego poprawiacza itd. Zwraca najlepszą znalezioną trasę, jej długość oraz historię najlepszego rozwiązania w kolejnych generacjach.

In [ ]:
def run_ga(dist, n_pop=50, n_gen=200, p_mut=0.05, p_cx=0.9,
           metoda_selekcji='turniej', metoda_krzyz='pmx', lokalna=True,
           ruchy_ls=['swap','two_opt','insertion'], elita=1, seed=None, turniej_k=3):
    if seed is not None:
        random.seed(seed); np.random.seed(seed)
    n = dist.shape[0]
    populacja = [losowa_trasa(n) for _ in range(n_pop)]
    najlepsza_hist = []
    for gen in range(n_gen):
        oceny = [dlugosc_trasy(ind, dist) for ind in populacja]
        fitness = [1.0/(o + 1e-9) for o in oceny]
        best_idx = int(np.argmin(oceny))
        najlepsza_hist.append(oceny[best_idx])
        nowa_pop = []
        ranked = sorted(zip(oceny, populacja), key=lambda x: x[0])
        # elita
        for i in range(elita):
            nowa_pop.append(deepcopy(ranked[i][1]))
        # generuj resztę
        while len(nowa_pop) < n_pop:
            if metoda_selekcji == 'turniej':
                p1 = selekcja_turniej(populacja, fitness, k=turniej_k)
                p2 = selekcja_turniej(populacja, fitness, k=turniej_k)
            else:
                p1 = selekcje[metoda_selekcji](populacja, fitness)
                p2 = selekcje[metoda_selekcji](populacja, fitness)
            # krzyżowanie
            if random.random() < p_cx:
                child = krzyzowania[metoda_krzyz](p1, p2)
            else:
                child = deepcopy(p1)
            # mutacja
            if random.random() < p_mut:
                child = mutacja_swap(child)
            # lokalne poprawienie
            if lokalna:
                child = lokalne_poprawienie(child, dist, ruchy_ls, proby=5)
            nowa_pop.append(child)
        populacja = nowa_pop
    oceny = [dlugosc_trasy(ind, dist) for ind in populacja]
    idx = int(np.argmin(oceny))
    return {'trasa': populacja[idx], 'dlugosc': oceny[idx], 'hist': najlepsza_hist}

## 8. Przykładowe uruchomienie i wykres konwergencji

Uruchom algorytm na macierzy `macierz` (jeśli wczytasz swoje dane z pliku Excel, podstaw tam `macierz`), a następnie narysuj wykres `hist` (najlepsze rozwiązanie w kolejnych generacjach).

In [ ]:
# Parametry do przykładu
param = {'n_pop': 50, 'n_gen': 150, 'p_mut': 0.05, 'p_cx': 0.9,
         'metoda_selekcji': 'turniej', 'metoda_krzyz': 'pmx', 'elita': 1}

wynik = run_ga(macierz, n_pop=param['n_pop'], n_gen=param['n_gen'],
               p_mut=param['p_mut'], p_cx=param['p_cx'],
               metoda_selekcji=param['metoda_selekcji'],
               metoda_krzyz=param['metoda_krzyz'],
               elita=param['elita'], seed=42, turniej_k=3, lokalna=True)

print('Najlepsza dlugosc:', wynik['dlugosc'])
plt.plot(wynik['hist'])
plt.xlabel('Generacja')
plt.ylabel('Długość najlepszej trasy')
plt.title('Konwergencja GA (przykład)')
plt.grid(True)
plt.show()

## 9. Eksperymenty: zapisywanie wyników i protokół

Zalecany format zapisu wyników to CSV, gdzie każdy wiersz to pojedyncze uruchomienie (run). Przykładowy kod do uruchamiania wielu powtórzeń i zapisywania wyników:

In [ ]:
def eksperyment_serie(dist, param_grid, powtorzenia=10, katalog_wyjscie='wyniki'):
    import os, csv
    os.makedirs(katalog_wyjscie, exist_ok=True)
    nazwa_csv = os.path.join(katalog_wyjscie, 'wyniki.csv')
    pola = ['run_id','seed','n_pop','n_gen','p_mut','p_cx','metoda_selekcji','turniej_k','metoda_krzyz','elita','lokalna_proby','czas_s','best_length','hist_json']
    with open(nazwa_csv, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=pola)
        writer.writeheader()
        run_id = 0
        for params in param_grid:
            for r in range(powtorzenia):
                seed = int(time.time()*1000) % 2**32 ^ r
                start = time.time()
                out = run_ga(dist,
                             n_pop=params.get('n_pop',50),
                             n_gen=params.get('n_gen',200),
                             p_mut=params.get('p_mut',0.05),
                             p_cx=params.get('p_cx',0.9),
                             metoda_selekcji=params.get('metoda_selekcji','turniej'),
                             metoda_krzyz=params.get('metoda_krzyz','pmx'),
                             elita=params.get('elita',1),
                             lokalna=params.get('lokalna',True),
                             seed=seed,
                             turniej_k=params.get('turniej_k',3))
                czas = time.time() - start
                hist_path = os.path.join(katalog_wyjscie, f'hist_{run_id}.json')
                with open(hist_path,'w') as hf:
                    json.dump(out['hist'], hf)
                writer.writerow({
                    'run_id': run_id,
                    'seed': seed,
                    'n_pop': params.get('n_pop',50),
                    'n_gen': params.get('n_gen',200),
                    'p_mut': params.get('p_mut',0.05),
                    'p_cx': params.get('p_cx',0.9),
                    'metoda_selekcji': params.get('metoda_selekcji','turniej'),
                    'turniej_k': params.get('turniej_k',3),
                    'metoda_krzyz': params.get('metoda_krzyz','pmx'),
                    'elita': params.get('elita',1),
                    'lokalna_proby': 5 if params.get('lokalna',True) else 0,
                    'czas_s': czas,
                    'best_length': out['dlugosc'],
                    'hist_json': hist_path
                })
                run_id += 1
    return nazwa_csv

# Przykład tworzenia siatki testowej (one-factor-at-a-time): testujemy 4 wartości n_gen
grid = [{'n_pop':50,'n_gen':v,'p_mut':0.05,'p_cx':0.9,'metoda_selekcji':'turniej','metoda_krzyz':'pmx'} for v in [100,300,600,1200]]
# Plik wynikowy = eksperyment_serie(macierz, grid, powtorzenia=5)  # UWAGA: może trwać długo dla dużych parametrów

## 10. Analiza wyników i przykładowe wykresy

Po zapisaniu `wyniki.csv` można wykonać wykresy: boxploty (porównanie wartości końcowych), wykresy konwergencji (uśrednione histy), heatmapy dla dwóch parametrów itd.

In [ ]:
# Przykład wczytania wyników i zrobienia prostego boxplotu (jeśli masz wyniki.csv)
# df = pd.read_csv('wyniki/wyniki.csv')
# import seaborn as sns
# plt.figure(figsize=(8,5))
# sns.boxplot(x='n_gen', y='best_length', data=df)
# plt.title('Porównanie końcowych wyników dla różnych n_gen')
# plt.show()

print('Jeśli wygenerujesz wyniki.csv możesz wkleić go tutaj i pomogę narysować wykresy.')

## 11. Porady praktyczne i dalsze kroki

- Zaczynaj od małych wartości (n_miast mały, n_pop i n_gen małe) by testować poprawność kodu.
- Pomiar czasu: miej kolumnę z czasem, bo większe parametry kosztują moc obliczeniową.
- Replikowalność: zapisuj seed dla każdego run.
- Eksperymentuj etapami: najpierw one-factor-at-a-time, potem siatki 2D dla najważniejszych parametrów.

Jeśli chcesz, mogę teraz:
- wygenerować ten notebook (zrobiłem to już) i podać link do pobrania,
- jeszcze dodać komórki z zadaniami do wykonania krok-po-kroku (np. checklistę),
- uruchomić przykładowe eksperymenty i wygenerować wykresy na podstawie Twojego pliku `Dane_TSP_127.xlsx`.
